In [1]:
import string
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import regularizers
from keras.models import load_model
import plotly.express as px
import pandas as pd

In [2]:
database = pd.read_csv("rsos_dream_data.csv")
df = database[["text_dream", "Male", "Animal", "Friends", "Family", "Dead&Imaginary",
                            "Aggression/Friendliness", "A/CIndex", "F/CIndex", "S/CIndex", "NegativeEmotions"]]

df['text_dream'] = df['text_dream'].apply(lambda x: x.lower())
df['text_dream'] = df['text_dream'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['text_dream'] = df['text_dream'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))
df['text_dream'] = df['text_dream'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Vectorize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text_dream'])
sequences = tokenizer.texts_to_sequences(df['text_dream'])

# Creating training and test data
X = pad_sequences(sequences, maxlen=100)
y = df[['Male', 'Animal', 'Friends', 'Family', 'Dead&Imaginary', 'Aggression/Friendliness', 'A/CIndex', 'F/CIndex',
        'S/CIndex', 'NegativeEmotions']]


C:\Users\Haochen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Haochen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Haochen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [21]:
class dream_analyse():
    def __init__(self):
        self.predictor = None

    def create_model(self):
        model = Sequential()
        model.add(Embedding(5000, 128, input_length=100))
        model.add(LSTM(128, kernel_regularizer=regularizers.L1(0.01)))
        model.add(Dense(10, activation='relu'))
        model.compile(loss='mae', optimizer='adam', metrics=['mae'])
        return model

    def train_model(self, X, Y):
        if self.predictor is None:
            self.predictor = self.create_model()
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
        self.predictor.fit(x_train, y_train, epochs=30, verbose=1, batch_size= 128, validation_data=(x_test, y_test),
          callbacks=[EarlyStopping(monitor='val_loss', patience=20)])
        self.predictor.save('model.h5')
        # Evaluate the model on the test set
        loss, accuracy = self.predictor.evaluate(x_test, y_test, verbose=0)
        print(f'Test loss: {loss:.3}')
        print(f'Test mse: {accuracy:.3}')

    def predict(self, text):
        if self.predictor == None:
            self.predictor = load_model('model.h5')
        text_token = tokenizer.texts_to_sequences([text])
        text_sequence = pad_sequences(text_token, maxlen=100)
        # Predict the scores for each category
        scores = self.predictor.predict(text_sequence)
        print(scores[0])
        plot_df = pd.DataFrame(dict( r = scores[0], theta = ["Male", "Animal", "Friends", "Family", "Dead&Imaginary",
                            "Aggression/Friendliness", "A/CIndex", "F/CIndex", "S/CIndex", "NegativeEmotions"]))
        fig = px.line_polar(plot_df, r='r', theta='theta', line_close=True)
        fig.show()








In [ ]:
dream_analyser = dream_analyse()
dream_analyser.train_model(X, y)



Epoch 1/30
 67/115 [================>.............] - ETA: 3s - loss: 15.5251 - mae: 0.2235  

In [16]:
text = df['text_dream'][1588]
print(text)
dream_analyser.predict(text)

i am in the m city house and looking out the pool room window at the little house i notice it is very ornate victorianlike and am surprised i dont remember it like that i see two officials and two girl singers at the platform the conference has begun i go through a door to see the exhibit table things are all piled up i start to spread things out so they can be seen i try to make an attractive display pr marketing is important i notice flyers for my book and also discuss the need for a sign on the door since the door is kept shut and the speakers and singers of my organization are in there ellie tells me a story about judy who is nearby listening she ellie says judy had lots of affairs once she even put the picture of one of the guys on her dresser and she was still married to her husband judy wasnt very proud of that she sits right in front of me as i am trying to watch the tv i sit up and put my head resting on hers or over to the right side we are very close george gives me a neckla